In [ ]:
import requests
from ipyleaflet import Map, LayerGroup, Marker, DivIcon, FullScreenControl, basemaps, TileLayer
from ipywidgets import Layout
from datetime import datetime, timezone
from psycopg2.extras import execute_values

plane_svg = """
<svg width="30px" height="30px" viewBox="0 0 512 512" version="1.1"
     xmlns="http://www.w3.org/2000/svg"
     style="transform: rotate({heading}deg); background-color: transparent;">
    <path fill="{color}"
          d="M448 336v-40L288 192V79.2c0-22.1-17.9-40-40-40s-40 17.9-40 40V192L48 296v40l160-48v113.6l-48 31.2V472l88-24 88 24v-39.2l-48-31.2V288l160 48z"
          stroke="white" stroke-width="10" stroke-linejoin="round"/>
</svg>
"""

In [ ]:
def initialize_database():
    conn = psycopg2.connect(
        host="localhost",
        port="10000",
        database="flight_data",
        user="admin",
        password="admin"
    )
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    print("Initializing Database...")
    cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb;")
    cur.execute("""
    CREATE TABLE IF NOT EXISTS observations (
        time        TIMESTAMPTZ NOT NULL,
        icao24      VARCHAR(10) NOT NULL,
        callsign    VARCHAR(20),
        location    GEOMETRY(POINT, 4326),
        altitude    DOUBLE PRECISION,
        velocity    DOUBLE PRECISION,
        heading     DOUBLE PRECISION,
        on_ground   BOOLEAN
    );
    """)
    try:
        cur.execute("SELECT create_hypertable('observations', 'time');")
        print("Hypertable created successfully.")
    except psycopg2.errors.DuplicateObject:
        print("Hypertable already exists, skipping.")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_icao_time ON observations (icao24, time DESC);")

    cur.close()
    conn.close()
    print("Database is ready!")

In [ ]:
def check_row_count():



    conn = psycopg2.connect(host="127.0.0.1", database="flight_data", user="admin", password="admin", port=10000)
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM observations;")
    count = cur.fetchone()[0]
    print(f"Current rows in DB: {count}")
    cur.close()
    conn.close()

In [ ]:
def save_to_db(planes):
    # Connection details from your docker-compose
    conn = psycopg2.connect(
        host="localhost",
        port="10000",
        database="flight_data",
        user="admin",
        password="admin"
    )
    cur = conn.cursor()

    execute_values(cur, """
        INSERT INTO observations (time, icao24, callsign, location, altitude, velocity, heading, on_ground)
        VALUES %s""",
        [(datetime.fromtimestamp(p['time'], tz=timezone.utc), p['icao24'], p['callsign'], f"SRID=4326;POINT({p['lon']} {p['lat']})",
          p['altitude'], p['velocity'], p['heading'], p['on_ground']) for p in planes])

    conn.commit()
    cur.close()
    conn.close()
    print(f"Successfully saved {len(planes)} rows to TimescaleDB.")

In [ ]:
def get_live_planes(username=None, password=None):
    # API Endpoint for all states
    url = "https://opensky-network.org/api/states/all"

    # Optional: Use auth to get higher rate limits (4000/day vs 100/day)
    auth = (username, password) if username and password else None

    try:
        response = requests.get(url, auth=auth, timeout=10)
        response.raise_for_status()
        data = response.json()

        states = data.get('states', [])
        timestamp = data.get('time')

        processed_planes = []

        for s in states:
            # We filter for planes that have valid lat/lon
            if s[5] is not None and s[6] is not None:
                processed_planes.append({
                    "time": timestamp,
                    "icao24": s[0],
                    "callsign": s[1].strip() if s[1] else "NONE",
                    "lon": s[5],
                    "lat": s[6],
                    "altitude": s[7], # Barometric altitude
                    "velocity": s[9],
                    "heading": s[10] or 0, # Default to 0 if null
                    "on_ground": s[8]
                })

        return processed_planes

    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

In [ ]:
planes = get_live_planes()
save_to_db(planes)
print(f"Found {len(planes)} planes with GPS data.")
check_row_count()

In [ ]:
# --- 1. Map Initialization (The "No-Repeat" Constraints) ---
# Center on London [51.5074, -0.1278]
m = Map(
    center=(51.5, -0.1),
    zoom=7,
    scroll_wheel_zoom=True,
    world_copy_jump=False,
    basemap=basemaps.OpenStreetMap.Mapnik,
    layout=Layout(width='100%', height='600px')
)

# Set hard limits so you can't pan into the "gray void" past the North/South poles
m.max_bounds = [(-90, -180), (90, 180)]

# Add a layer to hold planes
plane_layer = LayerGroup()
m.add_layer(plane_layer)
m.add_control(FullScreenControl())

# --- 2. Spatial Query Function ---
def get_visible_planes(south, west, north, east):
    """Queries TimescaleDB for planes strictly within the current view."""
    try:
        conn = psycopg2.connect(
            host="localhost", port="10000",
            database="flight_data", user="admin", password="admin"
        )
        cur = conn.cursor()

        query = """
        SELECT DISTINCT ON (icao24)
               icao24, callsign, ST_Y(location::geometry) as lat, ST_X(location::geometry) as lon,
               heading, on_ground
        FROM observations
        WHERE location && ST_MakeEnvelope(%s, %s, %s, %s, 4326);
        """

        cur.execute(query, (west, south, east, north))
        data = cur.fetchall()
        cur.close()
        conn.close()
        return data
    except Exception as e:
        print(f"DB Error: {e}")
        return []

# --- 3. The Update Logic ---
def update_view(change=None):
    """Triggered whenever the user moves or zooms."""
    bounds = m.bounds
    if not bounds or len(bounds) < 2:
        # Fallback for initial load
        south, west, north, east = 49.0, -2.0, 53.0, 2.0
    else:
        (south, west), (north, east) = bounds

    # Fetch and filter
    planes = get_visible_planes(south, west, north, east)
    print(f"found {len(planes)} planes with GPS data.")

    # Batch render
    new_markers = []
    for p in planes[:300]:
        icao, callsign, lat, lon, heading, on_ground = p

        # Plane icon with rotation
        color = "#28a745" if on_ground else "#007bff"
        icon_html = plane_svg.format(heading=heading or 0, color=color)
        icon = DivIcon(html=icon_html, bg_pos=[0, 0], icon_size=[30, 30])

        new_markers.append(Marker(location=(lat, lon), icon=icon, title=callsign, draggable=False))

    # Update the map in one shot
    plane_layer.clear_layers()
    plane_layer.layers = tuple(new_markers)

print(m.bounds)
# --- 4. Activation ---
m.observe(update_view, names='bounds')
display(m)

In [ ]:
# --- Configurare ---
DB_CONFIG = {
    "host": "127.0.0.1", "port": "10000",
    "database": "flight_data", "user": "admin", "password": "admin"
}

def clean_old_data():
    """Optional: Curata tabelul pentru a avea un demo clar."""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("TRUNCATE observations;") # Atentie! Sterge tot.
    conn.commit()
    conn.close()
    print("Baza de date a fost golită pentru test.")

def save_batch(planes):
    if not planes: return
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    query = """
        INSERT INTO observations (time, icao24, callsign, location, altitude, velocity, heading, on_ground)
        VALUES %s ON CONFLICT DO NOTHING
    """
    values = [
        (datetime.fromtimestamp(p['time'], tz=timezone.utc), p['icao24'], p['callsign'],
         p['lon'], p['lat'], p['altitude'], p['velocity'], p['heading'], p['on_ground'])
        for p in planes
    ]
    # Folosim template pentru PostGIS geometry
    execute_values(cur, query, values, template="""
    (%s, %s, %s, ST_SetSRID(ST_MakePoint(%s, %s), 4326), %s, %s, %s, %s)
    """)
    conn.commit()
    conn.close()

# --- Executia ---
# clean_old_data() # Decomenteaza daca vrei sa stergi datele anterioare
#
# ITERATIONS = 40
# FETCH_INTERVAL = 5 # Interval de colectare (secunde)
#
# print(f"Începem înregistrarea a {ITERATIONS} cadre...")
#
# for i in range(1, ITERATIONS + 1):
#     loop_start = time.time()
#
#     # 1. Fetch (Foloseste functia ta existenta get_live_planes)
#     planes = get_live_planes()
#     print(f"[{i}/{ITERATIONS}] Capturat {len(planes)} avioane.")
#
#     # 2. Save
#     save_batch(planes)
#
#     # 3. Wait (pentru a respecta intervalul de colectare)
#     elapsed = time.time() - loop_start
#     time.sleep(max(0, FETCH_INTERVAL - elapsed))
#
# print("✅ Înregistrare finalizată!")

In [1]:
import psycopg2
from ipyleaflet import Map, LayerGroup, Marker, DivIcon, basemaps
from ipywidgets import Layout, SelectionSlider, VBox, HTML
import time

# --- 1. Config ---
DB_CONFIG = {
    "host": "127.0.0.1", "port": "10000",
    "database": "flight_data", "user": "admin", "password": "admin"
}


style_html = HTML("""
<style>
    /* 1. Hides the label by default */
    .plane-label {
        display: none;
    }

    /* 2. Shows the label when you hover over the icon container */
    /* We target the class 'plane-icon' which you assign in Python */
    .plane-icon:hover .plane-label {
        border: none;
        width: 60px;'
        padding-up: 3px;'
        background-color: gray;'
        color: orange;'
        border-radius: 5px;'
        font-weight: bold;'
        text-align: center;
    }
</style>
""")

# --- 2. Helper to get Slider Options ---
def get_all_timestamps():
    """Fetch all unique timestamps to populate the slider."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        cur.execute("SELECT DISTINCT time FROM observations ORDER BY time ASC")
        rows = cur.fetchall()
        conn.close()
        # Return list of timestamps (formatted as strings if needed)
        return [r[0] for r in rows]
    except Exception as e:
        print(f"Error fetching timestamps: {e}")
        return []

# --- 3. The Tracker Logic (Simplified) ---
class TimeSliderTracker:
    def __init__(self, m, layer_group):
        self.map = m
        self.layer_group = layer_group
        self.active_markers = {}
        self.current_bounds = ()
        self.conn = None

        # Minified SVG Template
        self.svg_template = (
                             '<div class="plane-label">'
                             '{callsign} </div>'
                             '<svg width="24px" height="24px" viewBox="0 0 512 512" style="transform:rotate({heading}deg); transition: transform 0.2s linear; border: none;"><path fill="{color}" d="M448 336v-40L288 192V79.2c0-22.1-17.9-40-40-40s-40 17.9-40 40V192L48 296v40l160-48v113.6l-48 31.2V472l88-24 88 24v-39.2l-48-31.2V288l160 48z" stroke="white"/></svg>')

    def connect_db(self):
        if self.conn is None or self.conn.closed:
            try:
                self.conn = psycopg2.connect(**DB_CONFIG)
            except Exception as e:
                print(f"DB Error: {e}")

    def get_planes(self, timestamp):
        # 1. Handle bounds
        if not self.current_bounds:
            south, west, north, east = 35.0, -10.0, 60.0, 40.0
        else:
            ((south, west), (north, east)) = self.current_bounds

        try:
            # 2. Query
            if self.conn is None or self.conn.closed:
                self.connect_db()

            cur = self.conn.cursor()

            # Limit 300 for performance
            query = """
            SELECT icao24, callsign, ST_Y(location::geometry), ST_X(location::geometry), heading, on_ground
            FROM observations
            WHERE time = %s
            AND location && ST_MakeEnvelope(%s, %s, %s, %s, 4326)
            LIMIT 100;
            """
            cur.execute(query, (timestamp, west, south, east, north))
            data = cur.fetchall()
            cur.close()
            return data
        except Exception as e:
            print(f"Query Error: {e}")
            return []

    def update_frame(self, timestamp):
        """Optimized: O(1) Map Update instead of O(N) loops."""
        planes = self.get_planes(timestamp)
        current_icaos = set()

        for p in planes:
            icao, callsign, lat, lon, heading, on_ground = p
            current_icaos.add(icao)

            lat, lon = round(lat, 4), round(lon, 4)
            heading = int(heading) if heading else 0

            # --- 1. Update Existing Markers (Fast traitlet sync) ---
            if icao in self.active_markers:
                marker = self.active_markers[icao]

                if marker.location != (lat, lon):
                    marker.location = (lat, lon)

                if abs(marker.last_heading - heading) > 5:
                    color = "#28a745" if on_ground else "#007bff"
                    new_html = self.svg_template.format(heading=heading, color=color, callsign=callsign)
                    marker.icon = DivIcon(
                        html=icon_html,
                        icon_size=[0, 0],
                        class_name="plane-icon"
                    )
                    marker.last_heading = heading

            # --- 2. Create New Markers (Do NOT add to layer yet) ---
            else:
                color = "#28a745" if on_ground else "#007bff"
                icon_html = self.svg_template.format(heading=heading, color=color, callsign=callsign)
                new_icon = DivIcon(
                    html=icon_html,
                    icon_size=[0, 0],        # Your trick to hide the box
                    border=None,
                    class_name="plane-icon"
                )

                marker = Marker(location=(lat, lon), icon=new_icon, draggable=False, title=callsign)
                marker.last_heading = heading

                # Store in dictionary only
                self.active_markers[icao] = marker

        # --- 3. Batch Remove Dead Markers (Dictionary only) ---
        # Fast dictionary operation, no map interaction yet
        keys_to_remove = [k for k in self.active_markers if k not in current_icaos]
        for k in keys_to_remove:
            del self.active_markers[k]

        # --- 4. ATOMIC UPDATE (The Fix) ---
        # We update the layers tuple ONCE.
        # Ipyleaflet compares the new list IDs vs old list IDs.
        # It leaves existing markers alone, effectively handling add/remove instantly.
        self.layer_group.layers = tuple(self.active_markers.values())

        time.sleep(1)


    def update_bounds(self, change):
        self.current_bounds = change['new']
        for timestamp in get_all_timestamps():
            self.update_frame(timestamp)

# --- 4. Setup UI ---

# Load Data for Slider
timestamps = get_all_timestamps()

if not timestamps:
    print("No data found in DB!")
else:
    # Map Setup
    m = Map(
        center=(50, 10), zoom=6,
        basemap=basemaps.CartoDB.Positron,
        layout=Layout(width='100%', height='800px')
    )
    plane_layer = LayerGroup()
    m.add_layer(plane_layer)

    # Initialize Tracker
    tracker = TimeSliderTracker(m, plane_layer)
    m.observe(tracker.update_bounds, names='bounds')

    # Slider Setup
    slider = SelectionSlider(
        options=timestamps,
        description='Time:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        layout=Layout(width='90%')
    )

    # Label to show selected time clearly
    time_label = HTML(value=f"<b>Selected Time:</b> {timestamps[0]}")

    # Event Handler
    def on_slider_change(change):
        ts = change['new']
        time_label.value = f"<b>Selected Time:</b> {ts}"
        tracker.update_frame(ts)

    slider.observe(on_slider_change, names='value')

    # Initial Load
    on_slider_change({'new': timestamps[0]})

    # Display Layout
    ui = VBox([style_html, m, slider, time_label])
    display(ui)